In [ ]:
from __future__ import division
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import scipy as scp
import pylab as pyl

import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
if not os.path.isdir('DampedNewtonTesting_images'):
    os.makedirs('DampedNewtonTesting_images')

In [ ]:
"""To compute distance matrix"""
def distmat(x,y):
    
    return np.sum(x**2,0)[:,None] + np.sum(y**2,0)[None,:] - 2*x.transpose().dot(y)

"""To Normalise a vector"""
normalize = lambda a: a/np.sum(a)

"""To Compute P"""
def GetP(u,K,v):
    return u*K*(v.T)

def plotp(x, col,plt, scale=200, edgecolors="k"):
  return plt.scatter(x[0,:], x[1,:], s=scale, edgecolors=edgecolors,  c=col, cmap='plasma', linewidths=2)

In [ ]:
N=[400,400]

In [ ]:
x=np.random.rand(2,N[0])-0.5
theta =2*np.pi*np.random.rand(1,N[1])
r=0.8+.2*np.random.rand(1,N[1])
y=np.vstack((r*np.cos(theta),r*np.sin(theta)))

In [ ]:
import computational_OT

In [ ]:
rho=0.95
c1=0.05
DampedNewtonP=[]
results_DampedNewton  = []
times_DampedNewton    = []
Hessians_DampedNewton = []

#epsilons=[0.05,0.08,0.1]
epsilons=[0.2,0.3, 0.4, 0.5, 0.75, 1.0 ]
#epsilons=[0.3]
for eps in epsilons:
    # Line Search
    print("Damped Newton for epsilon="+str(eps)+":")    
    #Cost matrix
    C = distmat(x,y)

    # a and b
    a = normalize(np.ones(N[0]))
    a=a.reshape(a.shape[0],-1)
    b = normalize(np.ones(N[1]))
    b=b.reshape(b.shape[0],-1)

    #Epsilon 

    # epsilon = .05

    #Kernel
    K=np.exp(-C/eps)

    f,g=a,b

    print("Doing for (",N[0],N[1],").")
    print( " |- Iterating")
    start=time.time()
    eig_vector = np.hstack( (np.ones(a.shape[0]), -np.ones(b.shape[0])) )/np.sqrt( a.shape[0] + b.shape[0])
    eig_vector = np.reshape( eig_vector, (a.shape[0] + b.shape[0], 1) )
    reg_matrix=np.dot(eig_vector,eig_vector.T)
    Optimizer=computational_OT.DampedNewton(K,a,b,f,g,eps,rho,c1,reg_matrix)
    out=Optimizer._update(maxiter=100)
    results_DampedNewton.append(out)
    end=time.time()
    times_DampedNewton.append(end-start)
    print( " |- Computing P")
    DampedNewtonP.append(GetP(np.exp(out[0]/eps),K,np.exp(out[1]/eps)))
    print( " |- Recording (unstabilized) Hessian \n")

    mat  = -eps*Optimizer.Hessian
    diag = 1/np.sqrt( np.vstack( (a,b) ) ).flatten()
    mat = diag*mat*diag
    Hessians_DampedNewton.append( mat )


In [ ]:
plt.figure(figsize = (20,7))
plt.title("$$")
plt.title("$||P1 -a||_1+||P^T 1 -b||_1$")

for i in range(len(results_DampedNewton)):
  error=np.asarray(results_DampedNewton[i][2])+np.asarray(results_DampedNewton[i][3])
  plt.plot( error,label='Damped Newton for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)

plt.xlabel("Number of iterations")
plt.ylabel("Error in log-scale")
plt.legend()
plt.yscale( 'log')
plt.savefig("DampedNewtonTesting_images/ErrorLinesearchNewton.png")
plt.show()

print("\n Error plots can increase! The error is not the objective function!")

In [ ]:
plt.figure(figsize = (20,7))
plt.title("$$")

plt.title("Objective Function")

for i in range(len(results_DampedNewton)):
  plt.plot( np.asarray(results_DampedNewton[i][4]),label='Damped Newton for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)

plt.xlabel("Number of iterations")
plt.ylabel("Objective value")
plt.legend()
plt.savefig("DampedNewtonTesting_images/ObjectiveLineSearchNewton.png")
plt.show()


In [ ]:
plt.figure(figsize = (20,7))
plt.subplot(2,1,1),
plt.title("Alpha")

for i in range(len(results_DampedNewton)):
  plt.plot( np.asarray(results_DampedNewton[i][5]),label='Damped Newton for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)

plt.xlabel("Number of iterations")
plt.ylabel("Alpha in log-scale")
plt.legend()
# plt.yscale( 'log')
plt.savefig("DampedNewtonTesting_images/AlphaLineSearchNewton.png")
plt.show()



In [ ]:
def print_spectral_statistics(mat, stabilize=False):
    if stabilize:
        # Stabilizing largest and smallest eigenvalue
        min_vector = np.hstack( (np.ones(N[0]), -np.ones(N[1])) )
        max_vector = np.hstack( (np.ones(N[0]),  np.ones(N[1])) )
        norm = np.sqrt( N[0] + N[1] )
        min_vector = min_vector/norm
        max_vector = max_vector/norm
        min_vector = min_vector.reshape( (min_vector.shape[0], 1) )
        max_vector = max_vector.reshape( (max_vector.shape[0], 1) )
        #
        mat = mat + np.dot( min_vector, min_vector.T)
        mat = mat - np.dot( max_vector, max_vector.T)
    # endif
    eig, v = np.linalg.eig( mat )
    sorting_indices = np.argsort(eig)
    eig = eig[sorting_indices]
    v   = v[:, sorting_indices]
    
    #print( "Mean eigenvalue: ", np.mean(eig) )
    print( "List of smallest eigenvalues: ", eig[:10])
    print( "List of largest  eigenvalues: ", eig[-10:])
    min_index = np.argmin(eig)
    max_index = np.argmax(eig)
    min_value = eig[ min_index ]
    max_value = eig[max_index]
    min_vector = v[:, min_index]
    min_vector = min_vector/min_vector[0]
    max_vector = v[:,max_index]
    max_vector = max_vector/max_vector[0]
    condition_number = max_value/min_value
    # Test smallest and largest
    # print( "Min eigenvalue vector: ", min_vector)
    # print( "Max eigenvalue vector: ", max_vector)
    #
    #print( v[:,0]*np.sqrt( self.N1 + self.N2))
    #vector = v[:,0]
    #test = np.dot( result, vector)
    #print( np.linalg.norm(test) )
    #print("Min absolute eigenvalues: ", min_value)
    #print("Norm of v-1: ", np.linalg.norm(min_vector-eig_vector))
    print("Condition number: ", condition_number)
    # plt.hist( eig, 50)
    # plt.title( "Histogram of eigenvalues for Hessian")
    # plt.xlabel( "Eigenvalues")
    # plt.yscale( "log" )
    # plt.show()
    return eig,v

In [ ]:
eigs=[]
eigvecs=[]
for i in range(len(epsilons)):
#for i in range(len(epsilons)-1,len(epsilons)):
    eps = epsilons[i]
    print("Spectral statistics of Hessian for epsilon="+str(eps))
    Hessian = Hessians_DampedNewton[i]
    ev=print_spectral_statistics( Hessian, stabilize=False)
    eigs.append(ev[0])
    eigvecs.append(ev[1])
    print("")


In [ ]:
fig,ax=plt.subplots(figsize=(50,10),nrows=1, ncols=len(epsilons), sharey=True)
plt.title("Histogram of eigenvalues.")
for i in range(len(epsilons)):
    ax[i].hist( eigs[i], 50)
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]))
    ax[i].set_xlabel("Eigenvalues")
    ax[i].set_yscale( "log" )
plt.subplots_adjust(wspace=0,hspace=0)
plt.savefig("DampedNewtonTesting_images/eigenhistunstabilized.png")
plt.show()

## Heatmaps

In [ ]:

X=np.hstack((np.hstack((x[0,:],y[0,:]))[:,None],np.hstack((x[1,:],y[1,:]))[:,None]))
fig, ax = plt.subplots(figsize=(50,50),nrows=len(epsilons), ncols=4,sharey=True)
scale = 40

for i in range(len(epsilons)):

    im1 = ax[i][0].scatter(X.T[0,:], X.T[1,:], s=scale, edgecolors=(0,0,0,0),  c=eigvecs[i][0]/np.linalg.norm(eigvecs[i][0]), cmap='plasma', linewidths=2)
    fig.colorbar(im1,ax=ax[i][0])
    im = ax[i][1].scatter(X.T[0,:], X.T[1,:], s=scale, edgecolors=(0,0,0,0),  c=eigvecs[i][1]/np.linalg.norm(eigvecs[i][1]), cmap='plasma', linewidths=2)
    fig.colorbar(im1,ax=ax[i][1])
    im = ax[i][2].scatter(X.T[0,:], X.T[1,:], s=scale, edgecolors=(0,0,0,0),  c=eigvecs[i][2]/np.linalg.norm(eigvecs[i][2]), cmap='plasma', linewidths=2)
    fig.colorbar(im1,ax=ax[i][2])
    im1 = ax[i][3].scatter(X.T[0,:], X.T[1,:], s=scale, edgecolors=(0,0,0,0),  c=eigvecs[i][3]/np.linalg.norm(eigvecs[i][3]), cmap='plasma', linewidths=2)
    fig.colorbar(im1,ax=ax[i][3])
    ax[i][0].set_title("For smallest eigenvalue $\epsilon$= "+str(epsilons[i]))
    ax[i][1].set_title("For $2^{nd}$ smallest eigenvalue $\epsilon$= "+str(epsilons[i]))
    ax[i][2].set_title("For $3^{rd}$ smallest eigenvalue $\epsilon$= "+str(epsilons[i]))
    ax[i][3].set_title("For $4^{th}$ smallest eigenvalue $\epsilon$= "+str(epsilons[i]))


    
# plt.axis("off")
plt.xlim(np.min(X.T[0,:])-.1,np.max(X.T[0,:])+.1)
plt.ylim(np.min(X.T[1,:])-.1,np.max(X.T[1,:])+.1)
plt.subplots_adjust(wspace=0,hspace=0.09)
plt.savefig("DampedNewtonTesting_images/Eigenheatmaplowest.png")
plt.show();

In [ ]:

X=np.hstack((np.hstack((x[0,:],y[0,:]))[:,None],np.hstack((x[1,:],y[1,:]))[:,None]))
fig, ax = plt.subplots(figsize=(50,50),nrows=len(epsilons), ncols=4,sharey=True)
dot_scale = 40

for i in range(len(epsilons)):
    im1 = ax[i][0].scatter(X.T[0,:], X.T[1,:], s=dot_scale, edgecolors=(0,0,0,0),  c=eigvecs[i][-4]/np.linalg.norm(eigvecs[i][-4]), cmap='plasma', linewidths=2)
    fig.colorbar(im1,ax=ax[i][0])
    im1 = ax[i][1].scatter(X.T[0,:], X.T[1,:], s=dot_scale, edgecolors=(0,0,0,0),  c=eigvecs[i][-3]/np.linalg.norm(eigvecs[i][-3]), cmap='plasma', linewidths=2)
    fig.colorbar(im1,ax=ax[i][1])
    im1 = ax[i][2].scatter(X.T[0,:], X.T[1,:], s=dot_scale, edgecolors=(0,0,0,0),  c=eigvecs[i][-2]/np.linalg.norm(eigvecs[i][-2]), cmap='plasma', linewidths=2)
    fig.colorbar(im1,ax=ax[i][2])
    im1 = ax[i][3].scatter(X.T[0,:], X.T[1,:], s=dot_scale, edgecolors=(0,0,0,0),  c=eigvecs[i][-1]/np.linalg.norm(eigvecs[i][-1]), cmap='plasma', linewidths=2)
    fig.colorbar(im1,ax=ax[i][3])
    ax[i][3].set_title("For largest eigenvalue $\epsilon$= "+str(epsilons[i]))
    ax[i][2].set_title("For $2^{nd}$ largest eigenvalue $\epsilon$= "+str(epsilons[i]))
    ax[i][1].set_title("For $3^{rd}$ largest eigenvalue $\epsilon$= "+str(epsilons[i]))
    ax[i][0].set_title("For $4^{th}$ largest eigenvalue $\epsilon$= "+str(epsilons[i]))

# plt.axis("off")
plt.xlim(np.min(X.T[0,:])-.1,np.max(X.T[0,:])+.1)
plt.ylim(np.min(X.T[1,:])-.1,np.max(X.T[1,:])+.1)
plt.subplots_adjust(wspace=0,hspace=0.09)
plt.savefig("DampedNewtonTesting_images/Eigenheatmaplargest.png")
plt.show();

## Stabilized Histograms

In [ ]:
eigs=[]
eigvecs=[]
for i in range(len(epsilons)):
#for i in range(len(epsilons)-1,len(epsilons)):
    eps = epsilons[i]
    print("Spectral statistics of Hessian for epsilon="+str(eps))
    Hessian = Hessians_DampedNewton[i]
    ev=print_spectral_statistics( Hessian, stabilize=True)
    eigs.append(ev[0])
    eigvecs.append(ev[1])
    print("")


In [ ]:
fig,ax=plt.subplots(figsize=(50,10),nrows=1, ncols=len(epsilons),sharey=True)
plt.title("Histogram of eigenvalues.")
for i in range(len(epsilons)):
    ax[i].hist( eigs[i], 50)
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]))
    ax[i].set_xlabel("Eigenvalues")
    ax[i].set_yscale( "log" )
plt.subplots_adjust(wspace=0,hspace=0)
plt.savefig("DampedNewtonTesting_images/eigenhiststabilized.png")
plt.show()

In [ ]:

X=np.hstack((np.hstack((x[0,:],y[0,:]))[:,None],np.hstack((x[1,:],y[1,:]))[:,None]))
fig, ax = plt.subplots(figsize=(50,50),nrows=len(epsilons), ncols=4,sharey=True)
dot_scale = 40

for i in range(len(epsilons)):

    im = ax[i][0].scatter(X.T[0,:], X.T[1,:], s=dot_scale, edgecolors=(0,0,0,0),  c=eigvecs[i][0]/np.linalg.norm(eigvecs[i][0]), cmap='plasma', linewidths=2)
    fig.colorbar(im1,ax=ax[i][0])
    im = ax[i][1].scatter(X.T[0,:], X.T[1,:], s=dot_scale, edgecolors=(0,0,0,0),  c=eigvecs[i][1]/np.linalg.norm(eigvecs[i][1]), cmap='plasma', linewidths=2)
    fig.colorbar(im1,ax=ax[i][1])
    im = ax[i][2].scatter(X.T[0,:], X.T[1,:], s=dot_scale, edgecolors=(0,0,0,0),  c=eigvecs[i][2]/np.linalg.norm(eigvecs[i][2]), cmap='plasma', linewidths=2)
    fig.colorbar(im1,ax=ax[i][2])
    im1 = ax[i][3].scatter(X.T[0,:], X.T[1,:], s=dot_scale, edgecolors=(0,0,0,0),  c=eigvecs[i][3]/np.linalg.norm(eigvecs[i][3]), cmap='plasma', linewidths=2)
    fig.colorbar(im1,ax=ax[i][3])
    ax[i][0].set_title("For smallest eigenvalue $\epsilon$= "+str(epsilons[i]))
    ax[i][1].set_title("For $2^{nd}$ smallest eigenvalue $\epsilon$= "+str(epsilons[i]))
    ax[i][2].set_title("For $3^{rd}$ smallest eigenvalue $\epsilon$= "+str(epsilons[i]))
    ax[i][3].set_title("For $4^{th}$ smallest eigenvalue $\epsilon$= "+str(epsilons[i]))


    
# plt.axis("off")
plt.xlim(np.min(X.T[0,:])-.1,np.max(X.T[0,:])+.1)
plt.ylim(np.min(X.T[1,:])-.1,np.max(X.T[1,:])+.1)
plt.subplots_adjust(wspace=0,hspace=0.09)
plt.savefig("DampedNewtonTesting_images/Eigenheatmaploweststabilized.png")
plt.show();

In [ ]:

X=np.hstack((np.hstack((x[0,:],y[0,:]))[:,None],np.hstack((x[1,:],y[1,:]))[:,None]))
fig, ax = plt.subplots(figsize=(50,50),nrows=len(epsilons), ncols=4,sharey=True)
dot_scale = 40

for i in range(len(epsilons)):
    im1 = ax[i][0].scatter(X.T[0,:], X.T[1,:], s=dot_scale, edgecolors=(0,0,0,0),  c=eigvecs[i][-4]/np.linalg.norm(eigvecs[i][-4]), cmap='plasma', linewidths=2)
    fig.colorbar(im1,ax=ax[i][0])
    im1 = ax[i][1].scatter(X.T[0,:], X.T[1,:], s=dot_scale, edgecolors=(0,0,0,0),  c=eigvecs[i][-3]/np.linalg.norm(eigvecs[i][-3]), cmap='plasma', linewidths=2)
    fig.colorbar(im1,ax=ax[i][1])
    im1 = ax[i][2].scatter(X.T[0,:], X.T[1,:], s=dot_scale, edgecolors=(0,0,0,0),  c=eigvecs[i][-2]/np.linalg.norm(eigvecs[i][-2]), cmap='plasma', linewidths=2)
    fig.colorbar(im1,ax=ax[i][2])
    im1 = ax[i][3].scatter(X.T[0,:], X.T[1,:], s=dot_scale, edgecolors=(0,0,0,0),  c=eigvecs[i][-1]/np.linalg.norm(eigvecs[i][-1]), cmap='plasma', linewidths=2)
    fig.colorbar(im1,ax=ax[i][3])
    ax[i][3].set_title("For largest eigenvalue $\epsilon$= "+str(epsilons[i]))
    ax[i][2].set_title("For $2^{nd}$ largest eigenvalue $\epsilon$= "+str(epsilons[i]))
    ax[i][1].set_title("For $3^{rd}$ largest eigenvalue $\epsilon$= "+str(epsilons[i]))
    ax[i][0].set_title("For $4^{th}$ largest eigenvalue $\epsilon$= "+str(epsilons[i]))

# plt.axis("off")
plt.xlim(np.min(X.T[0,:])-.1,np.max(X.T[0,:])+.1)
plt.ylim(np.min(X.T[1,:])-.1,np.max(X.T[1,:])+.1)
plt.subplots_adjust(wspace=0,hspace=0.09)
plt.savefig("DampedNewtonTesting_images/Eigenheatmaplargeststabilzed.png")
plt.show();

### Testing New Regularization

In [ ]:
eigenvalues,eigenvectors=np.linalg.eig(Hessians_DampedNewton[0])
right_indices=np.where(eigenvalues>=1.5)
left_indices=np.where(eigenvalues<=0.5)
righteigvec=eigenvectors[right_indices]
lefteigvec=eigenvectors[left_indices]
reg_matrix=np.zeros((a.shape[0]+b.shape[0],a.shape[0]+b.shape[0]))
for i in righteigvec:
    # i=i/np.linalg.norm(i)
    # i=i/2
    i=i/np.sqrt(a.shape[0]+b.shape[0])
    reg_matrix=reg_matrix- np.dot(i[:,None],i[:,None].T)

for i in lefteigvec:
    # i=i/np.linalg.norm(i)
    i=i/np.sqrt(a.shape[0]+b.shape[0])
    reg_matrix=reg_matrix+np.dot(i[:,None],i[:,None].T)

In [ ]:
rho=0.95
c1=0.05
DampedNewtonP_=[]
results_DampedNewton_  = []
times_DampedNewton_    = []
Hessians_DampedNewton_ = []

#epsilons=[0.05,0.08,0.1]+-

# epsilons=[0.2,0.3, 0.4, 0.5, 0.75, 1.0 ]
epsilons=[0.3]
for eps in epsilons:
    # Line Search
    print("Damped Newton for epsilon="+str(eps)+":")    
    #Cost matrix
    C = distmat(x,y)
        
    # a and b
    a = normalize(np.ones(N[0]))
    a=a.reshape(a.shape[0],-1)
    b = normalize(np.ones(N[1]))
    b=b.reshape(b.shape[0],-1)

    #Epsilon 

    # epsilon = .05

    #Kernel
    K=np.exp(-C/eps)

    f,g=a,b

    print("Doing for (",N[0],N[1],").")
    print( " |- Iterating")
    start=time.time()
    Optimizer=computational_OT.DampedNewton(K,a,b,f,g,eps,rho,c1,reg_matrix)
    out=Optimizer._update(maxiter=100)
    results_DampedNewton_.append(out)
    end=time.time()
    times_DampedNewton_.append(end-start)
    print( " |- Computing P")
    DampedNewtonP_.append(GetP(np.exp(out[0]/eps),K,np.exp(out[1]/eps)))
    print( " |- Recording (unstabilized) Hessian \n")

    # mat  = -eps*Optimizer.Hessian
    # diag = 1/np.sqrt( np.vstack( (a,b) ) ).flatten()
    # mat = diag*mat*diag
    # Hessians_DampedNewton.append( mat )


In [ ]:
plt.figure(figsize = (20,7))
plt.title("$$")

plt.title("$||P1 -a||_1+||P^T 1 -b||_1$")

# for i in range(len(results_DampedNewton)):
error=np.asarray(results_DampedNewton[1][2])+np.asarray(results_DampedNewton[1][3])
plt.plot( error,label='Damped Newton for $\epsilon=$'+ str(epsilons[0]), linewidth = 2)
error=np.asarray(results_DampedNewton_[0][2])+np.asarray(results_DampedNewton_[0][3])
plt.plot( error,label='Damped Newton with regularisation for $\epsilon=$'+ str(epsilons[0]), linewidth = 2)

plt.xlabel("Number of iterations") 
plt.ylabel("Error in log-scale")
plt.legend()
plt.yscale( 'log')
plt.savefig("DampedNewtonTesting_images/ErrorLinesearchNewton.png")
plt.show()

print("\n Error plots can increase! The error is not the objective function!")

In [ ]:
eigs=[]
for i in range(len(epsilons)):
#for i in range(len(epsilons)-1,len(epsilons)):
    eps = epsilons[i]
    print("Spectral statistics of Hessian for epsilon="+str(eps))
    Hessian = Hessians_DampedNewton_[i]
    eigs.append(print_spectral_statistics( Hessian, stabilize=False))
    print("")


In [ ]:
fig,ax=plt.subplots(figsize=(50,10),nrows=1, ncols=len(epsilons), sharey=True)
plt.title("Histogram of eigenvalues.")
for i in range(len(epsilons)):
    ax[i].hist( eigs[i], 20)
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]))
    ax[i].set_xlabel("Eigenvalues")
    ax[i].set_yscale( "log" )
plt.subplots_adjust(wspace=0,hspace=0)
# plt.savefig("DampedNewtonTesting_images/eigenhistunstabilized.png")
plt.show()